## Link to article

This notebook is included in the documentation, where the interactive Plotly charts show up. See:
https://pegasystems.github.io/pega-datascientist-tools/Python/articles/vf_analysis.html

In [ ]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import plotly.io as pio
pio.renderers.default='notebook_connected'

import sys
sys.path.append('../../../')

# Value Finder analysis
Every Value Finder simulation populates a dataset, the pyValueFinder dataset.  This dataset contains a lot more information than is what is currently presented on screen.

The data held in this dataset can be analysed to uncover insights into your decision framework. 

pdstools has been updated to provide a notebook for some pre-configured analysis of the Value Finder dataset. This analysis can be used to supplement your Value Finder simulation whilst we add these features formally to the product.

In the data folder we’ve stored a copy of such a dataset, generated from an (internal) demo
application (CDHSample).

This page shows an example how the data can be used for additional analyses.

First, let’s look at the results as presented in Pega:

![Pega value finder screen](pegarun_8_6_0.png)

For the sample provided, the relevant action setting is 1.2%. There are 10.000 customers, 3491 without actions, 555 with only irrelevant actions and 5954 with at least one relevant action.

Now, let's import our class, read the data and recreate this view and supplement it with some advanced analysis of the pyValueFinder dataset. Just like with the ADMDatamart class, you can supply your own path and filename as such:
```python
ValueFinder(path = 'path-to-data', filename="Data-Insights_pyValueFinder_timestamp_GMT.zip")
```

If only a path is supplied, it will automatically look for the latest file. 
It is also possible to supply a dataframe as the 'df' argument directly, in which case it will use that instead. 
Lastly, there is now also an additional dataset in cdh tools, which is what we'll be using.

In [ ]:
from pdstools import ValueFinder, datasets
import polars as pl
vf = datasets.SampleValueFinder()

As we can see, it has found a file on the GitHub repo and imports it straight from there. 

Since there is only one dataset, the data is simply stored in the attribute 'df'. We heavily filter out the dataset for performance reasons, so the data will look like this:

In [ ]:
vf.df.head(5).collect()

This is already enough information to generate the same piechart as shown in platform, but to replicate the same values, we would need to compute the propensity threshold. In this case, the quantile of `0.052` of the propensity distribution seems to represent the same counts as in platform. Whilst we see the final pie chart after arbitration which correlates, it is also possible to view the same pie chart after each engagement policy stage. Simply call the `plotPieCharts()` function on the data:

In [ ]:
vf.plotPieCharts()

By hovering over the rightmost pie chart, you can see the numbers match up exactly to that shown in the value finder simulation. What's more, we don't just show the counts in the final arbitration stage, but also the counts of the eligibility, applicability, and the suitability stages. This view shows you the movement of customers from having at least one relevant action to only irrelevant actions and no actions over the application of the engagement policies. This will show you the most impactful stage of your policies.

Now, of course, if a customer is well served or not, depends heavily on what we consider to be well served. After the application of eligibility engagment policies we choose the relevant action setting. This is set at the 5th percentil of engagement policies. We can plot what that will look like as such, where the dotted line is that set threshold:

In [ ]:
vf.plotPropensityThreshold()

These different propensities represent the raw propensities from the models (pyModelPropensity), the propensities which may be overridden by the random control group (pyPropensity) and the final propensity from a prediction (FinalPropensity). In a prediction, Thompson Sampling may have been applied, smoothing the final distribution.

We can also look at the propensity distributions across the different stages. Simply call `plotPropensityDistribution()`.

In [ ]:
vf.plotPropensityDistribution()

So there is a spread out distribution of propensities - making the selection of the relevant action setting an important choice as this will influence whether customers are considered as having at least one relevant action or only irrelevant actions. 

While we can create this pie chart for one threshold, we can also do this for a range of them. To do this, simply supply three arguments to the `plotPieCharts()` function: `start`, `stop` and `step`. By default, these correspond to the propensity thresholds, but can also correspond to the quantiles with the 'method' argument. In the background, this will generate the aggregated counts per stage, which we can plot as such:

In [ ]:
vf.plotPieCharts(start=0.01, stop=0.5, step=0.01)

Note the slider at the bottom: playing around with this, you can easily see how choosing a different threshold changes the view of a customer. This makes intuitive sense: if you consider an action 'good' from a lower propensity threshold, then more customers will be well served than if you consider an action 'good' from a higher propensity threshold.

While this is a nice 'slice' of the distribution at a given threshold, we can also show a bit more information. Call `plotPropensityDistributionPerThreshold()` to show this same distribution, but then with the threshold on the x-axis. By default, it considers the threshold directly, but if you supply the `method` parameter to be `'quantile'`, then it will update to that instead.

In [ ]:
vf.plotDistributionPerThreshold()

In [ ]:
vf.plotDistributionPerThreshold(method='quantile')

One area to consider is how your action distribution changes through the stages. Simply call the `plotFunnelChart()` function for an overview of this funnel effect throughout each stage. As a rule of thumb, if there are only a few actions in each stage, this is not a good sign. If certain actions are completely filtered out from one stage to the next, it may also be a warning of agressive filtering. In this case, let’s also use the `‘query’` functionality to only look at actions in the `'Sales'` issue

In [ ]:
vf.plotFunnelChart('Action', query=pl.col('pyIssue')=='Sales')

Of course this is quite a lot of information. If instead we want to look at the distribution of *issues* over each stage, simply supply the `level` parameters as `'Issue'`:

In [ ]:
vf.plotFunnelChart('Issue')

Lastly, it may also be interesting to look at the distribution of groups over the different stages. Here, let's again filter on the `'Sales'` issue only.

In [ ]:
vf.plotFunnelChart('Group', query=pl.col('pyIssue')=='Sales')